## Deep Q-learning 

For more complex problems, a discrete state-action space may not be feasible. Instead we create a function approximation

$$Q_\theta(s,a)$$

Where $\theta$ are the function parameters.



We now seek to minimise the loss function

$$ L(\theta) = \mathbb{E}(y_k - Q_\theta(x_k, a_k))^2 $$

where

$$y_k = c_k + \lambda \min_a Q_\theta(x_{k+1}, a)$$

Therefore each time we perform our minimisation algorithm, we require a minibatch of quartets

$$e_k = (x_k, a_k, c_k, x_{k+1})$$

Like the Q-learning decscribed above, this is an off-policy method. Although this can be unstable, hence we may use

$$y_k = c_k + \lambda \min_a Q_{\theta'}(x_{k+1}, a)$$

where $Q_{\theta'}(x_{k}, a)$ tracks $Q_{\theta}(x_{k}, a)$ by periodically using $\theta' = \theta $.

## Actor-critic Methods

The above method has a notable problem, how can we gurantee performance with the presence of $\min_a$?

This is where we now introduce another neural network $\Pi_w$ that determines the suitable action given a state. $y_k$ is now given by 

$$y_k = c_k + \lambda \min_a Q_{\theta}(x_{k+1}, \Pi_w(x_{k+1}))$$

and the function to minimise for the actor is given by 

$$L_\Pi(w) = \mathbb{E}(Q_{\theta}(x, \Pi_w(x_))) $$
